In [71]:
import os, shutil

original_dataset_dir = '/Volumes/KESU/dog_vs_cat/train'
base_dir = '/Volumes/KESU/dog_vs_cat/dataset'
os.mkdir(base_dir)

train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)

validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)

train_cats_dir = os.path.join(train_dir, 'cats')
os.mkdir(train_cats_dir)

train_dogs_dir = os.path.join(train_dir, 'dogs')
os.mkdir(train_dogs_dir)

validation_cats_dir = os.path.join(validation_dir, 'cats')
os.mkdir(validation_cats_dir)

validation_dogs_dir = os.path.join(validation_dir, 'dogs')
os.mkdir(validation_dogs_dir)


In [72]:
#trainデータの８割を学習用、残りをバリデーション用として使用する
#eaach images 12500 
#train images 10000
#validation images 2500

fnames_cat = ['cat.{}.jpg'.format(i) for i in range(12500)]
fnames_dog = ['dog.{}.jpg'.format(i) for i in range(12500)]

In [73]:
#重複を許さず、猫ファイル名を取得
import random

files_cat = fnames_cat

# 乱数により移動先を振り分け

for f in files_cat:
    if random.random() >= 0.2: 
        # 学習用フォルダへ移動
        src = os.path.join(original_dataset_dir, f)
        dst = os.path.join(train_cats_dir,f)
        shutil.copy(src,dst)
    else:                      
        # dir2へ移動
        src = os.path.join(original_dataset_dir,f)
        dst = os.path.join(validation_cats_dir,f)
        shutil.copy(src,dst)

In [74]:
#重複を許さず、犬ファイル名を取得

# 乱数により移動先を振り分け

for f in fnames_dog:
    if random.random() >= 0.2: 
        # 学習用フォルダへ移動
        src = os.path.join(original_dataset_dir, f)
        dst = os.path.join(train_dogs_dir,f)
        shutil.copy(src,dst)
    else:                      
        # dir2へ移動
        src = os.path.join(original_dataset_dir,f)
        dst = os.path.join(validation_dogs_dir,f)
        shutil.copy(src,dst)

In [78]:
print('total training cat images', len(os.listdir(train_cats_dir)))
print('total training dog images', len(os.listdir(train_dogs_dir)))
print('///////////////////////////////////////////////////')
print('total validation cat images', len(os.listdir(validation_cats_dir)))
print('total validation dog images', len(os.listdir(validation_dogs_dir)))

total training cat images 10018
total training dog images 10007
///////////////////////////////////////////////////
total validation cat images 2482
total validation dog images 2493


In [79]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1,activation='softmax'))

In [80]:
from keras import optimizers

model.compile(loss='binary_crossentropy', 
                      optimizer= optimizers.Adam(lr=1e-4), 
                      metrics= ['acc'])

In [77]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

callback_cp = ModelCheckpoint(filepath='weights.{epoch:02d}.hdf5')

callback_es = EarlyStopping(monitor='val_acc', patience=2, mode='auto', verbose=1)

In [81]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
                                train_dir, 
                                target_size=(150, 150),
                                batch_size=32, 
                                class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
                                            validation_dir, 
                                            target_size=(150,150), 
                                            batch_size=32, 
                                            class_mode='binary')

Found 20025 images belonging to 2 classes.
Found 4975 images belonging to 2 classes.
